In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.amp
from torch.utils.checkpoint import checkpoint
import numpy as np
from contextlib import nullcontext
import math

torch.cuda.empty_cache()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# CONFIG – optimized for Colab A100 high-RAM (fast epochs)
triality = 3
dim = 384
latent_dim = 8
seq_len = 2048  # audio samples proxy
batch_size = 32
epochs = 20000
lr = 5e-5
use_amp = True
use_checkpoint = True

# Synthetic LibriSpeech audio proxy (real-like speech waveforms + noise/occlusion)
audio = []
for b in range(batch_size):
    t = torch.linspace(0, 10*math.pi, seq_len, device=device)
    wave = torch.sin(t * (20 + b % 10)) * 0.5  # "speech-like" frequency variation
    wave += torch.randn_like(wave) * 0.2  # noise
    wave = torch.clamp(wave, -1, 1)
    audio.append(wave.unsqueeze(-1))  # add feature dim

audio = torch.cat(audio, dim=0).to(device)  # (batch*seq_len, 1) -> reshape
audio = audio.view(batch_size, seq_len, -1)

# Project to dim
proj = nn.Linear(1, dim).to(device)
with torch.no_grad(): # Detach the input features from the graph for both models
    clean_data = proj(audio)

# Apply masking (40–70% missing samples — noise/dropouts)
missing_rate = torch.linspace(0.4, 0.7, batch_size, device=device).view(batch_size, 1, 1)
mask = torch.rand_like(clean_data) < missing_rate
real_data = clean_data.clone()
real_data[mask] = 0

target = clean_data

# E8 roots – precompute
def get_e8_roots():
    roots = []
    for i in range(8):
        for j in range(i+1, 8):
            for signs in [(1,1), (1,-1), (-1,1), (-1,-1)]:
                v = torch.zeros(8)
                v[i] = signs[0]; v[j] = signs[1]
                roots.append(v); roots.append(-v)
    for signs in range(1 << 8):
        v = torch.tensor([(1 if (signs & (1<<k)) else -1) for k in range(8)], dtype=torch.float32) * 0.5
        if bin(signs).count('1') % 2 == 0:
            roots.append(v); roots.append(-v)
    roots = torch.stack(roots[:240])
    return roots / roots.norm(dim=-1, keepdim=True)

e8_roots = get_e8_roots().to(device)

# Triality Cycle Block (detached pump scalar)
class AudioCycleBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.proj = nn.Linear(latent_dim, dim // triality, bias=False)
        self.register_buffer('roots', e8_roots)

    def forward(self, x, step):
        pos_emb = self.roots[torch.arange(x.shape[1], device=device) % 240]
        low_dim = self.proj(pos_emb)
        emb = low_dim.repeat(1, triality)
        with torch.no_grad():
            pump_scalar = 0.8 * math.sin(step * 0.006 * 2 * math.pi)
        pump = torch.full((1, x.shape[1], 1), pump_scalar, device=device)
        emb_broadcast = emb.unsqueeze(0)
        x_rot1 = x * (emb_broadcast.cos() + pump)
        x_rot2 = torch.roll(x_rot1, shifts=1, dims=1) * emb_broadcast.sin()
        x_rot3 = torch.roll(x_rot2, shifts=1, dims=1) * emb_broadcast.cos()
        fused = (x_rot1 + x_rot2 + x_rot3) / triality
        return fused

# Dummy cycle for ablation
class DummyCycle(nn.Module):
    def forward(self, x, step=None):
        return x

# Model with ablation support
class E8AudioFusion(nn.Module):
    def __init__(self, depth=32, use_triality=True):
        super().__init__()
        self.use_triality = use_triality
        self.cycle = AudioCycleBlock() if use_triality else DummyCycle()
        self.layers = nn.ModuleList([nn.MultiheadAttention(dim, triality if use_triality else 8, batch_first=True) for _ in range(depth)])
        self.norm = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, dim)

    def forward(self, x, step):
        x = self.cycle(x, step)
        for layer in self.layers:
            if use_checkpoint:
                attn, _ = checkpoint(layer, x, x, x, use_reentrant=False)
            else:
                attn, _ = layer(x, x, x)
            x = x + self.norm(attn)
        return x

# Models
model = E8AudioFusion(use_triality=True).to(device)
model_ablation = E8AudioFusion(use_triality=False).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=lr)
scaler = torch.amp.GradScaler('cuda') if use_amp else nullcontext()

opt_ablation = torch.optim.AdamW(model_ablation.parameters(), lr=lr)
scaler_ablation = torch.amp.GradScaler('cuda') if use_amp else nullcontext()

loss_fn = nn.MSELoss()

loss_hist = []
loss_abl_hist = []

for epoch in range(epochs):
    # --- Training for 'model' ---
    opt.zero_grad(set_to_none=True)
    with torch.amp.autocast(device_type='cuda', dtype=torch.float16) if use_amp else nullcontext():
        recon = model(real_data, epoch)
        loss = loss_fn(recon, target)

    scaler.scale(loss).backward() if use_amp else loss.backward()
    scaler.unscale_(opt) if use_amp else None
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1e6)
    scaler.step(opt) if use_amp else opt.step()
    scaler.update() if use_amp else None

    # --- Training for 'model_ablation' ---
    opt_ablation.zero_grad(set_to_none=True)
    with torch.amp.autocast(device_type='cuda', dtype=torch.float16) if use_amp else nullcontext():
        recon_abl = model_ablation(real_data, epoch)
        loss_abl = loss_fn(recon_abl, target)

    scaler_ablation.scale(loss_abl).backward() if use_amp else loss_abl.backward()
    scaler_ablation.unscale_(opt_ablation) if use_amp else None
    torch.nn.utils.clip_grad_norm_(model_ablation.parameters(), 1e6)
    scaler_ablation.step(opt_ablation) if use_amp else opt_ablation.step()
    scaler_ablation.update() if use_amp else None

    loss_hist.append(loss.item())
    loss_abl_hist.append(loss_abl.item())

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | Triality Loss {loss.item():.6f} | Ablation Loss {loss_abl.item():.6f}")

# Final Sigma Test
triality_mean = np.mean(loss_hist)
abl_mean = np.mean(loss_abl_hist)
std = np.std(loss_hist + loss_abl_hist)
sigma = (abl_mean - triality_mean) / std if std > 0 else 0

print(f"Final Sigma (Triality vs Ablation): {sigma:.2f} (higher = triality advantage)")

print("Sim complete — epochs + sigma test done")

Using device: cuda
Epoch 0 | Triality Loss 30.929083 | Ablation Loss 33.248138
Epoch 500 | Triality Loss 0.057787 | Ablation Loss 0.122760
Epoch 1000 | Triality Loss 0.051712 | Ablation Loss 0.112118
Epoch 1500 | Triality Loss 0.029844 | Ablation Loss 0.102177
Epoch 2000 | Triality Loss 0.024745 | Ablation Loss 0.096813
Epoch 2500 | Triality Loss 0.020120 | Ablation Loss 0.091855
Epoch 3000 | Triality Loss 0.018723 | Ablation Loss 0.085701
Epoch 3500 | Triality Loss 0.018299 | Ablation Loss 0.082985
Epoch 4000 | Triality Loss 0.017631 | Ablation Loss 0.075469
Epoch 4500 | Triality Loss 0.018833 | Ablation Loss 0.071593
Epoch 5000 | Triality Loss 0.017343 | Ablation Loss 0.067354
Epoch 5500 | Triality Loss 0.017486 | Ablation Loss 0.061201
Epoch 6000 | Triality Loss 0.017270 | Ablation Loss 0.056687
Epoch 6500 | Triality Loss 0.018546 | Ablation Loss 0.051620
Epoch 7000 | Triality Loss 0.015401 | Ablation Loss 0.049099
Epoch 7500 | Triality Loss 0.015076 | Ablation Loss 0.043938
Epoch 8